In [1]:
import boto3
import awswrangler as wr


In [2]:
session = boto3.Session(profile_name='datascientist')
s3 = session.client('s3')

#### Inflación

In [6]:
query = """
DROP TABLE IF EXISTS econ.inflacion;
"""

wr.athena.read_sql_query(
    query, 
    database="econ", 
    ctas_approach=False, 
    boto3_session=session
)

query = """

CREATE EXTERNAL TABLE econ.inflacion (
    fecha DATE,
    inflacion DOUBLE
) COMMENT 'Inflacion mensual'
ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe'
WITH SERDEPROPERTIES ('field.delim' = ',')
STORED AS INPUTFORMAT 'org.apache.hadoop.mapred.TextInputFormat' 
OUTPUTFORMAT 'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat'
LOCATION 's3://itam-analytics-grb/econ/raw/inflacion/'
TBLPROPERTIES ("skip.header.line.count"="1");

"""

wr.athena.read_sql_query(
    query, 
    database="econ", 
    ctas_approach=False, 
    boto3_session=session
)

""


#### Tasa de interés

In [ ]:
query = """
DROP TABLE IF EXISTS econ.tasa_de_interes;
"""

wr.athena.read_sql_query(
    query, 
    database="econ", 
    ctas_approach=False, 
    boto3_session=session
)

query = """

CREATE EXTERNAL TABLE econ.tasa_de_interes (
    fecha DATE,
    tasa_de_interes DOUBLE
) COMMENT 'Tasa e interés diaria'
ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe'
WITH SERDEPROPERTIES ('field.delim' = ',')
STORED AS INPUTFORMAT 'org.apache.hadoop.mapred.TextInputFormat' 
OUTPUTFORMAT 'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat'
LOCATION 's3://itam-analytics-grb/econ/raw/tasa_de_interes/'
TBLPROPERTIES ("skip.header.line.count"="1");

"""

wr.athena.read_sql_query(
    query, 
    database="econ", 
    ctas_approach=False, 
    boto3_session=session
)

""


#### Tipo de cambio

In [8]:
query = """
DROP TABLE IF EXISTS econ.tipo_de_cambio;
"""

wr.athena.read_sql_query(
    query, 
    database="econ", 
    ctas_approach=False, 
    boto3_session=session
)

query = """

CREATE EXTERNAL TABLE econ.tipo_de_cambio (
    fecha DATE,
    tipo_de_cambio DOUBLE
) COMMENT 'Tipo de cambio diario diaria'
ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe'
WITH SERDEPROPERTIES ('field.delim' = ',')
STORED AS INPUTFORMAT 'org.apache.hadoop.mapred.TextInputFormat' 
OUTPUTFORMAT 'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat'
LOCATION 's3://itam-analytics-grb/econ/raw/tipo_de_cambio/'
TBLPROPERTIES ("skip.header.line.count"="1");

"""

wr.athena.read_sql_query(
    query, 
    database="econ", 
    ctas_approach=False, 
    boto3_session=session
)

""


#### Creamos tabla mensual

In [ ]:

query = """
WITH 
    tipo_de_cambio_mensual AS (
        SELECT 
            DATE_TRUNC('month', fecha) AS fecha,
            AVG(tipo_de_cambio) AS tipo_de_cambio_mensual
        FROM econ.tipo_de_cambio
        GROUP BY 1
    ),

    tasa_de_interes_mensual AS (
        SELECT 
            DATE_TRUNC('month', fecha) AS fecha,
            AVG(tasa_de_interes) AS tasa_de_interes_mensual
        FROM econ.tasa_de_interes
        GROUP BY 1
    ),

    inflacion_mensual AS (
        SELECT 
            DATE_TRUNC('month', fecha) AS fecha,
            AVG(inflacion) AS inflacion_mensual
        FROM econ.inflacion
        GROUP BY 1
    )

SELECT 
    tdc.fecha,
    tdc.tipo_de_cambio_mensual,
    ti.tasa_de_interes_mensual,
    inf.inflacion_mensual
FROM tipo_de_cambio_mensual tdc
LEFT JOIN tasa_de_interes_mensual ti ON tdc.fecha = ti.fecha
LEFT JOIN inflacion_mensual inf ON tdc.fecha = inf.fecha
ORDER BY tdc.fecha;

"""

wr.athena.read_sql_query(
    query, 
    database="econ", 
    ctas_approach=False, 
    boto3_session=session
)

,fecha,tipo_de_cambio_mensual,tasa_de_interes_mensual,inflacion_mensual
0,1991-11-01,3.068377,NaN,19.720737
1,1991-12-01,3.070921,NaN,18.794623
2,1992-01-01,3.067723,NaN,17.947385
3,1992-02-01,3.062905,NaN,17.297151
4,1992-03-01,3.066768,NaN,16.824852
...,...,...,...,...
396,2024-11-01,20.345490,10.605735,4.548671
397,2024-12-01,20.266055,10.380720,4.212339
398,2025-01-01,20.549018,10.282523,3.585040
399,2025-02-01,20.457137,9.878884,3.773910
